<h1>Grabbing all Cryptos</h1>

In [ ]:
#Main Dependencies
import pandas as pd
import requests
from tkinter import *
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np

In [ ]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#Establish URL
url = 'https://coinmarketcap.com/'

In [ ]:
tokens = []

for i in range(0, 43, 1):
    loop_url = f'https://coinmarketcap.com/?page={i}'
    browser.visit(loop_url)
    
    for j in range(10):
        browser.execute_script("window.scrollBy(0,1000);")
        time.sleep(.5)
        
    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find('table', class_='cmc-table')
    trs = table.find_all('tr')

    for tr in trs:

        token = {}

        tds = tr.find_all('td')

        try:
            if tds[2].p is not None:
                coin_name = tds[2].p.text
                coin_code = tds[2].div.div.div.p.text
            else:
                coin_name = tds[2].find_all('span')[1].text
                coin_code = tds[2].find_all('span')[2].text

            coin_price = float(tds[3].text.replace(',','').strip('$'))
            coin_24h = float(tds[4].text.strip('%'))
            coin_7d = float(tds[5].text.strip('%'))
            
            try:
                coin_market_cap = float(tds[6].text.replace(',','').strip('$'))
            except Exception as e:
                coin_market_cap = None
                print(f'Warning: row [{i}] is missing market cap.')                

            # from Kick: use this as an example to isolate volume, which is sometimes blank
            try:
                coin_volume_usd = float(tds[7].find_all('p')[0].text.replace(',','').strip('$'))
                coin_volume_crypto_list = tds[7].find_all('p')[1].text.replace(',','').strip('$').split()
                coin_volume = float(coin_volume_crypto_list[0])
                coin_volume_token = coin_volume_crypto_list[1]
            except Exception as e:
                i = trs.index(tr)
                print(f'Warning: row [{i}] is missing volume.')
                coin_volume_usd = None
                coin_volume_crypto_list = None
                coin_volume = None
                coin_volume_token = None

            try:
                coin_circulating_supply_list = tds[8].text.replace(',','').strip('$').split()
                coin_circulating_supply = float(coin_circulating_supply_list[0])
                coin_circulating_supply_token = coin_circulating_supply_list[1]
            except Exception as e:
                print(f'Warning: row [{i}] is missing circulating supply.')                
                coin_circulating_supply_list = None
                coin_circulating_supply = None
                coin_circulating_supply_token = None
                

            # set up dictionary record
            token['token'] = coin_code
            token['name'] = coin_name
            token['price'] = coin_price
            token['24h'] = coin_24h
            token['7d'] = coin_7d
            token['market_cap'] = coin_market_cap
            token['volume_usd'] = coin_volume_usd
            token['volume'] = coin_volume
            token['circulating_supply'] = coin_circulating_supply

            # add to the list of dictionaries
            tokens.append(token)

        except Exception as e:
            i = trs.index(tr)
            #print(f'Error with index [{i}] of the tr.')
            continue
            
browser.quit()

In [ ]:
#Create DataFrame
token = pd.DataFrame(tokens)
token

In [ ]:
#Extract Master array of NAMES ONLY for all Cryptos
coins = token['name'].values
coins

<h1>Exchange Data - Database<h1>

In [ ]:
#Grab All URLS
links = []
for c in range(0,len(coins),1):
    lowercase = coins[c].lower()
    for r in ((" ", "-"), (".", "-")):
        lowercase = lowercase.replace(*r)
        exchange = f'https://coinmarketcap.com/currencies/{lowercase}/'
        
    #Appending
    links.append(exchange)

#Print Final Dictionary
#links

In [ ]:
list = []

for l in range(0,len(links),1):
    stats = f'https://coinmarketcap.com/currencies/{formed}/'
    analysis = requests.get(stats)
    broth = BeautifulSoup(analysis.text, 'html.parser')
    try:
        tables = broth.findAll('table')
        first_table_scrape = tables[0]
        first_table_tds = first_table_scrape.find_all('td')
        second_table_scrape = tables[1]
        second_table_tds = second_table_scrape.find_all('td')
        third_table_scrape = tables[2]
        third_table_tds = third_table_scrape.find_all('td')
        fourth_table_scrape = tables[3]
        fourth_table_tds = fourth_table_scrape.find_all('td')  
        fifth_table_scrape = tables[4]
        fifth_table_tds = fifth_table_scrape.find_all('td')
    except:
        continue
    texts = broth.select('div.about___1OuKY')
    code = broth.findAll('div', class_='sc-16r8icm-0 fCTazK nameHeader___27HU_')[0].find_all('small')[0].getText()
    
    table = {}

    #First Table
    price = first_table_tds[0].get_text()
    pricechange24hr = first_table_tds[1].find_all('span')[0].get_text()
    percentchange24hr = first_table_tds[1].find_all('span')[1].get_text()
    low24hr = first_table_tds[2].find_all('div')[0].get_text().split(' /')[0]
    high24hr = first_table_tds[2].find_all('div')[1].get_text().split(' /')[0]
    tradingvolume = first_table_tds[3].find_all('span')[0].get_text()
    tradingvolumepercdelta = first_table_tds[3].find_all('span')[1].get_text()
    marketdominance = first_table_tds[4].find_all('span')[0].get_text()
    marketrank = first_table_tds[5].get_text().split('#')[1]

    #Second Table
    marketcap = second_table_tds[0].find_all('span')[0].get_text()
    fullydiluted = second_table_tds[1].find_all('span')[0].get_text()

    #Third Table
    yestlow = third_table_tds[0].find_all('div')[0].get_text().split(' /')[0]
    yesthigh = third_table_tds[0].find_all('div')[1].get_text().split(' /')[0]
    yestopen = third_table_tds[1].find_all('div')[0].get_text().split(' /')[0]
    yestclose = third_table_tds[1].find_all('div')[1].get_text().split(' /')[0]
    yestvol = third_table_tds[3].get_text()

    #Fourth Table
    d7low = fourth_table_tds[0].find_all('div')[0].get_text().split(' /')[0]
    d7high = fourth_table_tds[0].find_all('div')[1].get_text().split(' /')[0]
    d30low = fourth_table_tds[1].find_all('div')[0].get_text().split(' /')[0]
    d30high = fourth_table_tds[1].find_all('div')[1].get_text().split(' /')[0]
    d90low = fourth_table_tds[2].find_all('div')[0].get_text().split(' /')[0]
    d90high = fourth_table_tds[2].find_all('div')[1].get_text().split(' /')[0]
    wk52low = fourth_table_tds[3].find_all('div')[0].get_text().split(' /')[0]
    wk52high = fourth_table_tds[3].find_all('div')[1].get_text().split(' /')[0]
    alltimehigh = fourth_table_tds[4].find_all('span')[0].get_text()
    alltimehighperc = fourth_table_tds[4].find_all('span')[1].get_text()
    alltimelow = fourth_table_tds[5].find_all('span')[0].get_text()
    alltimelowperc = fourth_table_tds[5].find_all('span')[1].get_text()
    roi = fourth_table_tds[6].findAll('p')[0].getText()

    #Fifth Table
    circsupp = fifth_table_tds[0].get_text()
    totsupp = fifth_table_tds[1].get_text()
    mxsupp = fifth_table_tds[2].get_text()

    #Narrative
    intro1 = texts[1].find_all('p')[0].get_text()
    intro2 = texts[1].find_all('p')[1].get_text()
    fullintro = f'{intro1}{intro2}'

    #Building the Dictionary
    table['Code'] = code
    table['Price'] = price
    table['24hr Price Change'] = pricechange24hr
    table['24hr % Change'] = percentchange24hr
    table['24hr Low'] = low24hr
    table['24hr High'] = high24hr
    table['24hr Trading Volume'] = tradingvolume
    table['24hr Trading Volume % Change'] = tradingvolumepercdelta
    table['Market Dominance'] = marketdominance
    table['Market Rank'] = marketrank
    table['Market Cap'] = marketcap
    table['Fully Diluted Market Cap'] = fullydiluted
    table['Yesterday Low'] = yestlow
    table['Yesterday High'] = yesthigh
    table['Yesterday Open'] = yestopen
    table['Yesterday Close'] = yestclose
    table['Yesterday Volume'] = yestvol
    table['7 Day Low'] = d7low
    table['7 Day High'] = d7high
    table['30 Day Low'] = d30low
    table['30 Day High'] = d30high
    table['90 Day Low'] = d90low
    table['90 Day High'] = d90high
    table['52 Week Low'] = wk52low
    table['52 Week High'] = wk52high
    table['All Time High'] = alltimehigh
    table['All Time High %'] = alltimehighperc
    table['All Time Low'] = alltimelow
    table['All Time Low %'] = alltimelowperc
    table['Return on Investment'] = roi
    table['Cirulating Supply'] = circsupp
    table['Total Supply'] = totsupp
    table['Max Supply'] = mxsupp
    table["About"] = fullintro
    table['Link'] = stats

    #Appending
    tables.append(table)

#Print Final Dictionary
#tables

cryptobio = pd.DataFrame(tables)
cryptobio

In [ ]:
#Export to CSV
cryptobio.to_csv('ExchangeInfo.csv', index=False, header=True)

In [ ]:
#Export to JSON
import csv
import json

def make_json(csvFilePath, jsonFilePath):

    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:

            key = rows['Date']
            data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
csvFilePath = r'ExchangeInfo.csv'
jsonFilePath = r'ExchangeInfo.json'
 
make_json(csvFilePath, jsonFilePath)

<h1>Historical Data - DataBase</h1>

In [ ]:
#NEW URL - Creating Date List
dates = 'https://coinmarketcap.com/historical/'

In [ ]:
#Cooking in the Kitchen
select = requests.get(dates)
pho = BeautifulSoup(select.text, 'html.parser')
#pho

In [ ]:
#Pull history of all dates
links = pho.find_all('div',class_='cmc-bottom-margin-2x')[0].find_all('a',class_='historical-link cmc-link')
#links

In [ ]:
#Create DataFrame
dates = []

for l in range(36,len(links),1):
    
    date = {}
    
    nums=links[l].get('href').split("/")[2]
    
    dates.append(nums)
    
#dates

In [ ]:
#Grab All URLS
links = []
for d in range(0,len(dates),1):
    history = f'https://coinmarketcap.com/historical/{dates[d]}/'
        
    #Appending
    links.append(history)

#Print Final Dictionary
#links

In [ ]:
logs = []

for l in range(0,len(links),1):
    story = requests.get(links[l])
    chowder = BeautifulSoup(story.text, 'html.parser')
    exhibits = chowder.findAll('tbody')
    try:
        itinerary = exhibits[0]
        itinerary_trs = itinerary.find_all('tr')
    except:
        continue
    for w in range(0,len(itinerary),1):

        tds = itinerary_trs[w].find_all('td')

        log = {}

        rank = tds[0].find_all('div')[0].get_text()
        name = tds[1].find_all('a')[0].getText()
        symbol = tds[2].find_all('div')[0].get_text()
        marketcap = tds[3].find_all('p')[0].getText()
        price = tds[4].find_all('a')[0].getText()
        circsupp = tds[5].find_all('div')[0].get_text()
        vol24h = tds[6].find_all('a')[0].getText()
        p1h = tds[7].find_all('div')[0].get_text()
        p24h = tds[8].find_all('div')[0].get_text()
        p7d = tds[9].find_all('div')[0].get_text()

        log['Date'] = links[l].split('/')[4]
        log['Rank'] = rank
        log['Name'] = name
        log['Symbol'] = symbol
        log['Market Cap'] = marketcap
        log['Price'] = price
        log['Circulating Supply'] = circsupp
        log['1 Hour Percent Change'] = p1h
        log['24 Hour Percent Change'] = p24h
        log['7 Day Percent Change'] = p7d


        logs.append(log)

#logs
textbook = pd.DataFrame(logs)
textbook['Date'] = pd.to_datetime(textbook['Date'], format='%Y%m%d',errors='coerce')
textbook

In [ ]:
#Export to CSV
textbook.to_csv('History.csv', index=False, header=True)

In [ ]:
#Export to JSON
import csv
import json

def make_json(csvFilePath, jsonFilePath):

    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:

            key = rows['Date']
            data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
csvFilePath = r'History.csv'
jsonFilePath = r'History.json'
 
make_json(csvFilePath, jsonFilePath)